In [1]:
import scipy
import numpy as np

In [2]:
import pyriemann
def RM_M_G(data):
    RM_M=pyriemann.estimation.Covariances('scm').fit_transform(data)
    return RM_M

In [3]:
def LogmTranfer(mySPDdata):
    Dimension_1=len(mySPDdata)
    Dimension_2=len(mySPDdata[1])
    myRMlog=np.zeros((Dimension_1,Dimension_2,Dimension_2))
    for i in range(len(mySPDdata)):
        myRMlog[i]=scipy.linalg.logm(mySPDdata[i], disp=True)#
    return myRMlog

In [4]:
# output the cov for 3D SPD data (N, n, n)
def cov_3D(samples):
    size = samples[0].shape
    # m*n matrix
    mean = np.zeros(size)

    for s in samples:
        mean = mean + s

    # get the mean of all samples
    mean /= float(len(samples))

    # n*n matrix
    cov_row = np.zeros((size[1],size[1]))
    for s in samples:
        diff = s - mean;
        cov_row = cov_row + np.dot(diff.T, diff)
    cov_row /= float(len(samples))
    return cov_row

In [5]:
def PCA2D_BYCOV(cov_row, row_top):#  return X, Z
    row_eval, row_evec = np.linalg.eigh(cov_row)
    # select the top t evals
    sorted_index = np.argsort(row_eval)
    # using slice operation to reverse
    X = row_evec[:,sorted_index[:-row_top-1 : -1]]
    Z=X
    return X, Z

In [6]:
def PCA_DATA(X_THIS,Z_THIS,data,wantaftersize):
    data_R=np.zeros((len(data),wantaftersize,wantaftersize))
    for sz in range(len(data)):
        data_R[sz] = np.dot(Z_THIS.T, np.dot(data[sz], X_THIS))
    return data_R

In [7]:
#size_af,meanall,g_all,cov_all
def IB2DPCA_Update(mean_before,g_squre_before, length_before ,samples_update):
    '''update samples are 2d matrices'''  
    size2 = samples_update[0].shape
    # m*n matrix
    sum2 = np.zeros(size2)
    g2=np.zeros(size2)
    for sp in samples_update:
        sum2 = sum2 + sp
        g2=g2+np.dot(sp,sp.T)
    mean2=sum2 / float(len(samples_update))
    # meanall1=(sum+sum2)/ float(len(samples)+len(samples_update))
    meanall=length_before/(float(length_before+len(samples_update)))*mean_before+1/((float(length_before+len(samples_update))))*len(samples_update)*mean2
    g_all=g_squre_before+g2
    cov_all=g_all-(length_before+len(samples_update))*(np.dot(meanall,meanall.T))
    cov_all=cov_all/float(len(samples_update)+length_before)
    size_af=len(samples_update)+length_before

    return size_af,meanall,g_all,cov_all


In [8]:

def get_top_eigenpairs(cov_matrix, n_top):
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    top_eigenvalues = eigenvalues[-n_top:][::-1]
    top_eigenvectors = eigenvectors[:, -n_top:][:, ::-1]

    return top_eigenvalues, top_eigenvectors

In [9]:
# Generate a random array with shape (100, 22, 1000)
np.random.seed(42)
offline_data = np.random.rand(100, 22, 1000)
offline_data=RM_M_G(offline_data)

offline_data=LogmTranfer(offline_data)
size,mean,g,s=IB2DPCA_Update(mean_before=0,g_squre_before=0,length_before=0,samples_update=offline_data)
print('size  ',size,'  mean  ',mean.shape,' g',g.shape,'    s',s.shape)
X,Z=PCA2D_BYCOV(s,18)

offline_data_R=PCA_DATA(X,Z,offline_data,18)
print(offline_data_R.shape)

s_batch=cov_3D(offline_data)

size   100   mean   (22, 22)  g (22, 22)     s (22, 22)
(100, 18, 18)


In [10]:
v,ve=get_top_eigenpairs(s,4)
v

array([0.02610867, 0.02554657, 0.02544378, 0.02482334])

In [11]:
v,ve=get_top_eigenpairs(s_batch,4)
v

array([0.02610867, 0.02554657, 0.02544378, 0.02482334])

In [12]:
# Generate a random array with shape (100, 22, 1000)
np.random.seed(24)

online_data = np.random.rand(100, 22, 1000)
online_data=RM_M_G(online_data)
online_data=LogmTranfer(online_data)

#IB2DPCA ONLY using online_data
size2,mean2,g2,s2=IB2DPCA_Update(mean_before=mean,g_squre_before=g,length_before=size,samples_update=online_data)
print('size  ',size2,'  mean  ',mean2.shape,' g',g2.shape,'    s',s2.shape)
X2,Z2=PCA2D_BYCOV(s2,18)

online_data_R=PCA_DATA(X2,Z2,online_data,18)
print(online_data_R.shape)

#using offline_data and online_data
s_batch2=cov_3D(np.concatenate([offline_data,online_data],axis=0))

size   200   mean   (22, 22)  g (22, 22)     s (22, 22)
(100, 18, 18)


In [13]:
v,ve=get_top_eigenpairs(s2,4)
v

array([0.02476323, 0.02434726, 0.02416686, 0.02375289])

In [14]:
v,ve=get_top_eigenpairs(s_batch2,4)
v

array([0.02476323, 0.02434726, 0.02416686, 0.02375289])